In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import os
import skimage.filters

from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
images = []
labels = []
SIZE = 224

for directory_path in glob.glob("/content/drive/MyDrive/ML_IPData7/*"):
    label = directory_path.split("/")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        images.append(img)
        labels.append(label)

Rifle
Golf cart


In [ ]:
images = np.array(images)
type(labels)
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
labelencoder = LabelEncoder()
labelencoder.fit(labels)
enc_labels = labelencoder.transform(labels)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, enc_labels, test_size = 0.20, random_state = 10)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
import tensorflow_hub as hub

base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_5           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,211,201 (99.99 MB)

 Trainable params: 2,623,489 (10.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics =['accuracy'])

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 433ms/step - accuracy: 0.9137 - loss: 0.3665


In [ ]:
print("initial loss: {:.2f}".format(loss))
print("initial accuracy: {:.2f}".format(accuracy))

initial loss: 0.39
initial accuracy: 0.88


In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_test, y_test))

Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 16s 296ms/step - accuracy: 0.8258 - loss: 0.3993 - val_accuracy: 0.9796 - val_loss: 0.0560
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.9572 - loss: 0.1155 - val_accuracy: 0.9796 - val_loss: 0.0369
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.9711 - loss: 0.0629 - val_accuracy: 0.9932 - val_loss: 0.0232
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - accuracy: 0.9857 - loss: 0.0484 - val_accuracy: 0.9864 - val_loss: 0.0212
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 111ms/step - accuracy: 0.9879 - loss: 0.0338 - val_accuracy: 0.9932 - val_loss: 0.0218
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - accuracy: 0.9927 - loss: 0.0269 - val_accuracy: 0.9864 - val_loss: 0.0260
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - accuracy: 0.9928 - loss: 0.0183 - val_accuracy: 0.9932 - val_loss: 0.0217
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step - accuracy: 0.9978 - loss: 0.0141 - val_accuracy: 0.98

In [ ]:
print("Training Loss:", history.history['loss'][-1]*100)
print("Validation Loss:", history.history['val_loss'][-1]*100)
print("\n\n")
print("Training Accuracy:", history.history['accuracy'][-1]*100)
print("Validation Accuracy:", history.history['val_accuracy'][-1]*100)




Training Loss: 0.7940421812236309
Validation Loss: 3.040287084877491



Training Accuracy: 99.82905983924866
Validation Accuracy: 98.63945841789246
